#   Homework 3
## Sentiment analysis using Neural Networks

Total: 50 Points


In this homework we will perform sentiment analysis using a few simple Neural Network based architectures.
For this problem we use the IMDB Large Movie Review Dataset. The dataset contains 25,000 highly polar movie reviews for both train and test dataset, each with 12,500 positive (greater than equal to 7/10 rating) and 12,500 negative reviews(less than equal to 4/10 rating). 

Use "https://keras.io/" for keras documentation. Please use Python 3. GPU is not required but it will help improve the training speed for each problem.

Please save the notebook with your cell outputs. You will not be graded if your outputs are not present below the homework cell. Also note your outputs will be unique since you will be using your the last numbers of your uni as your random seed (In the third cell). Make sure you submit this iPython file, with the saved outputs. The submission format must be 'hw3/hw3.ipynb'. You will not submit any other files. If you do save your model weights, you will not submit them. You will however, make sure your model weights do get saved in the 'weights' folder and can be retrieved from there as well.

Please fill your details below.



Name: Colby Wise

Uni: cjw2165

Email: cjw2165@columbia.edu


In [7]:
from os import listdir
import random
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Embedding, Dense, Dropout, Reshape, Merge, BatchNormalization, TimeDistributed, Lambda, Activation, LSTM, Flatten, Convolution1D,Conv1D, GRU, MaxPooling1D
from keras.regularizers import l2
from keras.callbacks import Callback, ModelCheckpoint, EarlyStopping
#from keras import initializers
from keras import backend as K
from keras.optimizers import SGD
from keras.optimizers import Adadelta
from keras.utils import np_utils
from keras.preprocessing import sequence
from keras import optimizers
import numpy as np

In [8]:


#we retrieve train and test file names

train_dir = "./aclImdb/train/"
test_dir = "./aclImdb/test/"
tr_review = [re_filename for re_filename in listdir(train_dir)]
te_review = [re_filename for re_filename in listdir(test_dir)]

#we initialize the train and test arrays

tr_X = []
tr_Y = []
te_X = []
te_Y = []

#we arrange the reviews into the train and test arrays 

for review_file in tr_review:
    f_review = open(train_dir+review_file, "r")
    str_review = f_review.readline()
    str_review = " ".join(str_review.split(' '))
    tr_X.append(str_review)
    y_truth = int (review_file.split('.')[0].split('_')[1])
    if y_truth>=7:
        tr_Y.append(1)
    else:
        tr_Y.append(0)
        
for review_file in te_review:
    f_review = open(test_dir+review_file, "r")
    str_review = f_review.readline()
    str_review = " ".join(str_review.split(' '))
    te_X.append(str_review)
    y_truth = int (review_file.split('.')[0].split('_')[1])
    if y_truth>=7:
        te_Y.append(1)
    else:
        te_Y.append(0)
        

We will now create the validation set from the train set

use the last 4 numbers of your uni for the seed value seed to ensure all answers remain unique.

In [9]:
#replace 2 (SEED) with the last 4 numbers of your Uni
#Uni: cjw2165
SEED = 2165
seed_counter = 0
while(1):

    shuffle_combine = list(zip(tr_X, tr_Y))
    random.seed(SEED+seed_counter)
    seed_counter+=1
    random.shuffle(shuffle_combine)

    tr_X, tr_Y = zip(*shuffle_combine)

    val_X = tr_X[:5000]
    val_Y = tr_Y[:5000]

    counter = 0
    for label in val_Y:
        counter+=label

    print (counter)
    print (seed_counter)
    if(counter>2400 and counter <2600):
        tr_X = tr_X[5000:]
        tr_Y = tr_Y[5000:]
        break

2529
1


In [10]:


print("Length of Train review set : " + str(len(tr_X)))
print("Length of Train label set : " + str(len(tr_Y)))
print("Length of Validation review set : " + str(len(val_X)))
print("Length of Validation label set : " + str(len(val_Y)))
print("Length of Test review set : " + str(len(te_X)))
print("Length of Test label set : " + str(len(te_Y)))
print("*****************************************")
print("Some sample Reviews Train sets and their labels")
print(tr_X[0][:150])
print(tr_Y[0])
print(tr_X[1][:150])
print(tr_Y[1])
print(tr_X[2][:150])
print(tr_Y[2])
print(tr_X[3][:150])
print(tr_Y[3])
print(tr_X[4][:150])
print(tr_Y[4])

Length of Train review set : 20000
Length of Train label set : 20000
Length of Validation review set : 5000
Length of Validation label set : 5000
Length of Test review set : 25000
Length of Test label set : 25000
*****************************************
Some sample Reviews Train sets and their labels
This type of plot really does have a lot of potential, but it was butchered here. Honestly, I sensed the cheese element in the beginning, but I though
0
SPOILER ALERT ! ! ! Personally I don't understand why Pete did not help to save Williams life,I mean that would be great to know why William was motiv
0
Tough guys, sexy women, lots of swearing, and a most unconvincing monster that rises from the depths of a polluted lake. You'd think "Monster" would b
0
One of the first OVA's ("original video animation") I ever bought, this still has to be one of my favourite anime titles. A cyberpunk sci-fi action co
1
I was very disappointed in this movie. Plotwise it was weak bordering on silly: Souls

In [11]:
#we collect all the reviews from train validation and test set to generate 
texts = []
texts += tr_X 
texts += te_X 
texts += val_X
len(texts)



#we clip the sentence length to first 250 words. 
MAX_SEQUENCE_LENGTH = 250

#length of vocab, Tokenizer will only use vocab_len most common words
vocab_len = 25000

#we tokenize the texts and convert all the words to tokens
tokenizer = Tokenizer(num_words=vocab_len)
tokenizer.fit_on_texts(texts)

token_tr_X = tokenizer.texts_to_sequences(tr_X)
token_te_X = tokenizer.texts_to_sequences(te_X)
token_val_X = tokenizer.texts_to_sequences(val_X)

#to ensure all reviews have the same length, we pad the smaller reviews with 0, 
#and cut the larger reviews to a max length 
#(we clip from the top, as the end of the reviews generally have a conclusion which provides better features)
x_train = sequence.pad_sequences(token_tr_X, maxlen=MAX_SEQUENCE_LENGTH)
x_test = sequence.pad_sequences(token_te_X, maxlen=MAX_SEQUENCE_LENGTH)
x_val = sequence.pad_sequences(token_val_X, maxlen=MAX_SEQUENCE_LENGTH)


#changes the labels to one-hot encoding
y_train = np_utils.to_categorical(tr_Y)
y_test = np_utils.to_categorical(te_Y)
y_val = np_utils.to_categorical(val_Y)


In [12]:
print('X_train shape:', x_train.shape)
print('X_test shape:', x_test.shape)
print('X_val shape:', x_val.shape)

print('y_train shape:', y_train.shape)
print('y_test shape:', y_test.shape)
print('y_val shape:', y_val.shape)


print("*****************************************")
print("Tokenized Reviews Train sets and their labels")
print(x_train[0][:20])
print(y_train[0])
print()
print(x_train[1][:20])
print(y_train[1])
print()
print(x_train[2][:20])
print(y_train[2])
print()
print(x_train[3][:20])
print(y_train[3])
print()
print(x_train[4][:20])
print(y_train[4])
print()

X_train shape: (20000, 250)
X_test shape: (25000, 250)
X_val shape: (5000, 250)
y_train shape: (20000, 2)
y_test shape: (25000, 2)
y_val shape: (5000, 2)
*****************************************
Tokenized Reviews Train sets and their labels
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[ 1.  0.]

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[ 1.  0.]

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[ 1.  0.]

[   0    0    0    0    0   27    4    1   86  212  373  839   10  123 1226
   11  130   45    5   26]
[ 0.  1.]

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[ 1.  0.]



********************************************

As you can see the reviews have now been transformed into indices to tokenized vocabulary and the labels have been converted to one-hot encoding. We can now go ahead and feed these sequences to Neural Network Models.

********************************************

# Part A

Building your first model (5 Points)

Construct this sequential model using Keras :

![title](img/model1.jpg)

In [13]:
print('Build model...')

## implement model here

model = Sequential()
model.add(Embedding(input_dim=25000, output_dim=128, input_length=250))
model.add(Flatten( ) )
model.add(Dense(200))
model.add(Activation('relu'))
model.add(Dense(units=2))
model.add(Activation('softmax'))
model.compile(loss='categorical_crossentropy',
optimizer='Adam',
metrics=['accuracy'])

model.summary()

print("Model Built")

Build model...
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 250, 128)          3200000   
_________________________________________________________________
flatten_1 (Flatten)          (None, 32000)             0         
_________________________________________________________________
dense_1 (Dense)              (None, 200)               6400200   
_________________________________________________________________
activation_1 (Activation)    (None, 200)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 2)                 402       
_________________________________________________________________
activation_2 (Activation)    (None, 2)                 0         
Total params: 9,600,602
Trainable params: 9,600,602
Non-trainable params: 0
___________________________________________________

In [14]:
print('Train...')
model.fit(x_train, y_train,
          batch_size=32,
          epochs=4,
          validation_data=(x_val, y_val),
          verbose = 1,
         shuffle = True)

Train...
Train on 20000 samples, validate on 5000 samples
Epoch 1/4
20000/20000 [==============================] - 56s 3ms/step - loss: 0.3963 - acc: 0.8058 - val_loss: 0.2760 - val_acc: 0.8888
Epoch 2/4
20000/20000 [==============================] - 56s 3ms/step - loss: 0.0411 - acc: 0.9861 - val_loss: 0.4266 - val_acc: 0.8622
Epoch 3/4
20000/20000 [==============================] - 56s 3ms/step - loss: 0.0027 - acc: 0.9996 - val_loss: 0.5191 - val_acc: 0.8662
Epoch 4/4
20000/20000 [==============================] - 56s 3ms/step - loss: 1.7571e-04 - acc: 1.0000 - val_loss: 0.5658 - val_acc: 0.8660


# Part B

Stacking Fully Connected Layers (5 points)

Construct this sequential model using Keras :

![title](img/model2.jpg)

In [15]:
print('Build model...')

## implement model here

model = Sequential()
model.add(Embedding(input_dim=25000, output_dim=128, input_length=250))
model.add(Flatten( ) )
model.add(Dense(200))
model.add(Activation('relu'))
model.add(Dense(200))
model.add(Activation('relu'))
model.add(Dense(units=2))
model.add(Activation('softmax'))
model.compile(loss='categorical_crossentropy',
               optimizer='Adam',
            metrics=['accuracy'])

model.summary()

print("Model Built")

Build model...
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 250, 128)          3200000   
_________________________________________________________________
flatten_2 (Flatten)          (None, 32000)             0         
_________________________________________________________________
dense_3 (Dense)              (None, 200)               6400200   
_________________________________________________________________
activation_3 (Activation)    (None, 200)               0         
_________________________________________________________________
dense_4 (Dense)              (None, 200)               40200     
_________________________________________________________________
activation_4 (Activation)    (None, 200)               0         
_________________________________________________________________
dense_5 (Dense)              (None, 2)                 402   

In [16]:
print('Train...')
model.fit(x_train, y_train,
          batch_size=32,
          epochs=4,
          validation_data=(x_val, y_val),
          verbose = 1,
         shuffle = True)

Train...
Train on 20000 samples, validate on 5000 samples
Epoch 1/4
20000/20000 [==============================] - 57s 3ms/step - loss: 0.4016 - acc: 0.8006 - val_loss: 0.2903 - val_acc: 0.8782
Epoch 2/4
20000/20000 [==============================] - 56s 3ms/step - loss: 0.0434 - acc: 0.9848 - val_loss: 0.4570 - val_acc: 0.8452
Epoch 3/4
20000/20000 [==============================] - 56s 3ms/step - loss: 0.0040 - acc: 0.9988 - val_loss: 0.8088 - val_acc: 0.8526
Epoch 4/4
20000/20000 [==============================] - 56s 3ms/step - loss: 0.0139 - acc: 0.9952 - val_loss: 0.6902 - val_acc: 0.8380


# Part C

Using LSTMS based networks(5 Points) 

Construct this sequential model using Keras :

![title](img/model3.jpg)

In [17]:
print('Build model...')

## implement model here

model = Sequential()

model.add(Embedding(input_dim=25000, output_dim=128, input_length=250))
model.add(LSTM(128))
model.add(Dense(128))
model.add(Activation('relu'))
model.add(Dense(units=2))
model.add(Activation('softmax'))
model.compile(loss='categorical_crossentropy',
              optimizer='Adam',
              metrics=['accuracy'])

model.summary()

print("Model Built")

Build model...
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, 250, 128)          3200000   
_________________________________________________________________
lstm_1 (LSTM)                (None, 128)               131584    
_________________________________________________________________
dense_6 (Dense)              (None, 128)               16512     
_________________________________________________________________
activation_6 (Activation)    (None, 128)               0         
_________________________________________________________________
dense_7 (Dense)              (None, 2)                 258       
_________________________________________________________________
activation_7 (Activation)    (None, 2)                 0         
Total params: 3,348,354
Trainable params: 3,348,354
Non-trainable params: 0
___________________________________________________

In [18]:

print('Train...')
model.fit(x_train, y_train,
          batch_size=32,
          epochs=5,
          validation_data=(x_val, y_val),
          verbose = 1,
         shuffle = True)

Train...
Train on 20000 samples, validate on 5000 samples
Epoch 1/5
20000/20000 [==============================] - 157s 8ms/step - loss: 0.4548 - acc: 0.7867 - val_loss: 0.3515 - val_acc: 0.8548
Epoch 2/5
20000/20000 [==============================] - 156s 8ms/step - loss: 0.2157 - acc: 0.9176 - val_loss: 0.3265 - val_acc: 0.8778
Epoch 3/5
20000/20000 [==============================] - 156s 8ms/step - loss: 0.1209 - acc: 0.9571 - val_loss: 0.3445 - val_acc: 0.8748
Epoch 4/5
20000/20000 [==============================] - 155s 8ms/step - loss: 0.0831 - acc: 0.9708 - val_loss: 0.4156 - val_acc: 0.8486
Epoch 5/5
20000/20000 [==============================] - 154s 8ms/step - loss: 0.0555 - acc: 0.9813 - val_loss: 0.5607 - val_acc: 0.8628


# Part D

Adding Pretrained Word Embeddings(10 Points)

Construct this sequential model using Keras :

Correction: The Embedding Layer Dimension (1st box) is 300, not 128.

![title](img/model4.jpg)

In [19]:
import codecs

#dimension of Glove Embeddings.
EMBEDDING_DIM = 300

word_index = tokenizer.word_index
print('Found %s unique tokens' % len(word_index))

#load glove embeddings
gembeddings_index = {}
with codecs.open('glove.42B.300d.txt', encoding='utf-8') as f:
    for line in f:
        values = line.split(' ')
        word = values[0]
        gembedding = np.asarray(values[1:], dtype='float32')
        gembeddings_index[word] = gembedding
#
f.close()
print('G Word embeddings:', len(gembeddings_index))

# nb_words contains the total length of vocab
nb_words = len(word_index) +1

#get glove embeddings for each word in tokenizer.
#g_word_embedding_matrix holds the embeddings dictionary
g_word_embedding_matrix = np.zeros((nb_words, EMBEDDING_DIM))

for word, i in word_index.items():
    gembedding_vector = gembeddings_index.get(word)
    if gembedding_vector is not None:
        g_word_embedding_matrix[i] = gembedding_vector
        
#total words in the tokenizer not in Embedding matrix
print('G Null word embeddings: %d' % np.sum(np.sum(g_word_embedding_matrix, axis=1) == 0))



Found 124252 unique tokens
G Word embeddings: 1917494
G Null word embeddings: 35772


In [21]:
print('Build model...')

## implement model here

model = Sequential()

## to use the glove embeddings, your embedding layer would take the vocab size as input dimension, 
## Glove embedding dimension as the output dimsion
## and you will provide the  embedding dictionary as the 'weights' parameter (!important) to the embedding layer.

model.add(Embedding(input_dim=g_word_embedding_matrix.shape[0],
          output_dim=300,
          weights=[g_word_embedding_matrix],
          input_length=250,
          trainable=False))

model.add(LSTM(128, recurrent_dropout=0.2))
model.add(Dropout(0.2))
model.add(Dense(128))
model.add(Activation('relu'))
model.add(Dense(units=2))
model.add(Activation('softmax'))
model.compile(loss='categorical_crossentropy',
              optimizer='Adam',
              metrics=['accuracy'])

## compille it here according to instructions

model.summary()

print("Model Built")

Build model...
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_4 (Embedding)      (None, 250, 300)          37275900  
_________________________________________________________________
lstm_2 (LSTM)                (None, 128)               219648    
_________________________________________________________________
dropout_1 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_8 (Dense)              (None, 128)               16512     
_________________________________________________________________
activation_8 (Activation)    (None, 128)               0         
_________________________________________________________________
dense_9 (Dense)              (None, 2)                 258       
_________________________________________________________________
activation_9 (Activation)    (None, 2)                 0     

In [22]:
print('Train...')
model.fit(x_train, y_train,
          batch_size=32,
          epochs=5,
          validation_data=(x_val, y_val),
          verbose = 1,
         shuffle = True)

Train...
Train on 20000 samples, validate on 5000 samples
Epoch 1/5
20000/20000 [==============================] - 207s 10ms/step - loss: 0.4757 - acc: 0.7767 - val_loss: 0.3411 - val_acc: 0.8556
Epoch 2/5
20000/20000 [==============================] - 203s 10ms/step - loss: 0.3127 - acc: 0.8694 - val_loss: 0.2877 - val_acc: 0.8858
Epoch 3/5
20000/20000 [==============================] - 203s 10ms/step - loss: 0.2652 - acc: 0.8918 - val_loss: 0.2789 - val_acc: 0.8878
Epoch 4/5
20000/20000 [==============================] - 203s 10ms/step - loss: 0.2309 - acc: 0.9072 - val_loss: 0.2635 - val_acc: 0.8964
Epoch 5/5
20000/20000 [==============================] - 201s 10ms/step - loss: 0.1944 - acc: 0.9223 - val_loss: 0.2827 - val_acc: 0.8910


# Dont attempt this

Stacking LSTM layers

Unfortunately it takes very long to train, be aware we can stack LTMSs over each other like this.
This requires bottom LSTM to return a sequences instead instead of single vector, which becomes input for the top LSTM.


![title](img/model5.jpg)

# Part E

Using Convolutional Networks (10 points)

Construct the model, shown below. Use the same loss functions and optimizers as before

Correction: The Embedding Layer Dimension (1st box) is 300, not 128.

![title](img/model6.jpg)

In [23]:
print('Build model...')

## implement model here
from keras.layers import Conv1D, MaxPooling1D

model = Sequential()
model.add(Embedding(input_dim=g_word_embedding_matrix.shape[0],
          output_dim=300,
          weights=[g_word_embedding_matrix],
          input_length=250,
          trainable=False))

model.add(Conv1D(128, 3))
model.add(Dropout(0.2))
model.add(Activation('relu'))
     
model.add(Conv1D(64, 3))
model.add(Dropout(0.2))
model.add(Activation('relu'))

model.add(Conv1D(32, 3))
model.add(Dropout(0.2))
model.add(Activation('relu'))

model.add(Flatten())

model.add(Dense(256))
model.add(Dropout(0.2))
model.add(Activation('relu'))

model.add(Dense(2))
model.add(Activation('softmax'))


model.compile(loss='categorical_crossentropy',
              optimizer='Adam',
              metrics=['accuracy'])

model.summary()

print("Model Built")

Build model...
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_5 (Embedding)      (None, 250, 300)          37275900  
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 248, 128)          115328    
_________________________________________________________________
dropout_2 (Dropout)          (None, 248, 128)          0         
_________________________________________________________________
activation_10 (Activation)   (None, 248, 128)          0         
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 246, 64)           24640     
_________________________________________________________________
dropout_3 (Dropout)          (None, 246, 64)           0         
_________________________________________________________________
activation_11 (Activation)   (None, 246, 64)           0     

In [24]:
print('Train...')
model.fit(x_train, y_train,
          batch_size=32,
          epochs=5,
          validation_data=(x_val, y_val),
          verbose = 1,
         shuffle = True)

Train...
Train on 20000 samples, validate on 5000 samples
Epoch 1/5
20000/20000 [==============================] - 77s 4ms/step - loss: 0.4654 - acc: 0.7577 - val_loss: 0.3358 - val_acc: 0.8678
Epoch 2/5
20000/20000 [==============================] - 75s 4ms/step - loss: 0.3119 - acc: 0.8691 - val_loss: 0.2872 - val_acc: 0.8816
Epoch 3/5
20000/20000 [==============================] - 76s 4ms/step - loss: 0.2679 - acc: 0.8909 - val_loss: 0.2808 - val_acc: 0.8876
Epoch 4/5
20000/20000 [==============================] - 76s 4ms/step - loss: 0.2300 - acc: 0.9076 - val_loss: 0.2864 - val_acc: 0.8840
Epoch 5/5
20000/20000 [==============================] - 75s 4ms/step - loss: 0.1977 - acc: 0.9208 - val_loss: 0.2713 - val_acc: 0.8892


# Part F

Model constructed : (5 points)

Test Accuracy Over 87.5%: (5 Points)

Bonus: Min(10, Square of (test_score - 88%))

Create your best model, use Validation score to judge your best model and check accuracy on test set


In [ ]:
print('Build model...')

## implement model here
from keras.layers import Conv1D, MaxPooling1D

model = Sequential()
model.add(Embedding(input_dim=g_word_embedding_matrix.shape[0],
          output_dim=300,
          weights=[g_word_embedding_matrix],
          input_length=250,
          trainable=False))

model.add(Conv1D(256, 3))
model.add(Dropout(0.4))
model.add(Activation('relu'))
     
model.add(Conv1D(128, 3))
model.add(Dropout(0.2))
model.add(Activation('relu'))

model.add(Conv1D(64, 3))
model.add(Dropout(0.2))
model.add(Activation('relu'))

model.add(Flatten())

model.add(Dense(256))
model.add(Dropout(0.2))
model.add(Activation('relu'))

model.add(Dense(2))
model.add(Activation('softmax'))


model.compile(loss='categorical_crossentropy',
              optimizer='Adam',
              metrics=['accuracy'])

model.summary()

print("Model Built")

Build model...
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_5 (Embedding)      (None, 250, 300)          37275900  
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 248, 128)          115328    
_________________________________________________________________
dropout_2 (Dropout)          (None, 248, 128)          0         
_________________________________________________________________
activation_10 (Activation)   (None, 248, 128)          0         
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 246, 64)           24640     
_________________________________________________________________
dropout_3 (Dropout)          (None, 246, 64)           0         
_________________________________________________________________
activation_11 (Activation)   (None, 246, 64)           0     

You can keep saving models with different names in model_name, 

so you can retrieve their weights again for testing, you dont have to retrain 
(You would have to initialize the model definition again).

In [24]:
# wt_dir = "./weights/"
# model_name = 'model1_best'
# early_stopping = EarlyStopping(monitor='val_acc', patience=2)
# bst_model_path = wt_dir + model_name + '.h5'
# model_checkpoint = ModelCheckpoint(bst_model_path, monitor='val_acc', save_best_only=True, save_weights_only=True)

print('Train...')
model_1.fit(x_train, y_train,
          batch_size=32,
          epochs=6,
          validation_data=(x_val, y_val),
          verbose = 1,
         shuffle = True,
#          callbacks=[early_stopping, model_checkpoint])
        callbacks=[early_stopping])

print('Train...')
model.fit(x_train, y_train,
          batch_size=32,
          epochs=5,
          validation_data=(x_val, y_val),
          verbose = 1,
         shuffle = True)

Train...
Train on 20000 samples, validate on 5000 samples
Epoch 1/5
20000/20000 [==============================] - 77s 4ms/step - loss: 0.4654 - acc: 0.7577 - val_loss: 0.3358 - val_acc: 0.8678
Epoch 2/5
20000/20000 [==============================] - 75s 4ms/step - loss: 0.3119 - acc: 0.8691 - val_loss: 0.2872 - val_acc: 0.8816
Epoch 3/5
20000/20000 [==============================] - 76s 4ms/step - loss: 0.2679 - acc: 0.8909 - val_loss: 0.2808 - val_acc: 0.8876
Epoch 4/5
20000/20000 [==============================] - 76s 4ms/step - loss: 0.2300 - acc: 0.9076 - val_loss: 0.2864 - val_acc: 0.8840
Epoch 5/5
20000/20000 [==============================] - 75s 4ms/step - loss: 0.1977 - acc: 0.9208 - val_loss: 0.2713 - val_acc: 0.8892


If you plan on using Ensemble averaging, feel free to edit the code below or add multiple models.

Make sure they get saved and can be retrieved when executing serially.

In [25]:
#model.load_weights(bst_model_path)
scores = model.evaluate(x_test, y_test, verbose=1)
print("Accuracy: %.2f%%" % (scores[1]*100))

25000/25000 [==============================] - 34s 1ms/step
Accuracy: 88.61%


# Part G

Explain how Dense, LSTM and Convolution Layers work.

Explain Relu, Dropout, and Softmax work.

Analyze the architectures you constructed, with the accuracies you achieved and the training time it took. 

What are some insights you gained with these experiments? 

(5 Points)


Dense, LSTM, CNN --
    The Dense layer is a fully connected layer where every layer node is connected to each node in the following layer. LSTM is a form of recurrent nueral net (RNN) where the state space is halved with one being the memory and the other working memory. CNN or convolution nueral networks come from computer vision/image recognition field of DL and are great for learning complex features by taking local ordering patterns into account - great for ngrams.
    
Relu, Dropout, Softmax --
    Relu is a not linear activation applied during feed-forward step that takes the max(1,0) of the input transformation creating a nonlinear output. Dropout can be applied before or after activation but essentially turns the node in the layer off such that the node is not used during the forward pass forcing the network to not rely on that nodes contribution. Softmax transforms the linear prediction into a probability [0,1] summing to 1 in the multi-score case. 
    
Architectures Analysis --
    In Part A and B we used sequential models and I noticed that adding another Dense layer had no impact on the validation accuracy. This is because the Dense layer was simply doing another linear transformation. Training time in A/B were four times faster than LSTM. Additionally accuracy was the best in LSTM(s) (~2% greater than sequential) particularly part D. Training epochs to reach 100% accuracy on train need to be increased in C/D which means training LSTMs takes longer which is inline with what we saw. CNN was fairly quick (2x sequential and 50% of lstm) and performed inline in terms of validation accuracy with LSTM albeit with more consistent generalization.
    
Insights --
    Dropout had a modestly more noticeable effect on CNNs than sequential/LSTM consistent with literature that dropout in LSTMs has marginal effect. Furthermore, I found the best form of regularization to be decaying the learning rate and/or early stopping given the models tended to hit minimum loss around 4-5 epochs thus training for longer epochs required decaying learning rates or early stopping. I also found it interesting that wider (more activation in hidden layer) and deeper (more hd layers) did not necessarily lead to signification accuracy increases. In other sub-fields of deep learning like vision deeper and to some extent wider CNNs for instance have led to major accuracy increases such as in LeNet, Deep Dream from Google. Here the results did not immediately transcend. Pre-trained word embeddings lead to significant increases. I would be curious to experiment with different pre-trainings in future. 
    